In [ ]:
import numpy as np
import pandas as pd
import ddm
import os 
import helper
import models

In [ ]:
def get_model_measures(model, condition):
    sol = model.solve(condition)
    mean_rt_wait = np.sum(sol.pdf_err()*model.t_domain())*model.dt / sol.prob_error()
    return condition["tta_condition"], condition["d_condition"], sol.prob_correct(), \
           sol.mean_decision_time(), mean_rt_wait

def get_model_rt_distr(model, condition, kind="cdf"):
    sol = model.solve(condition)
    return pd.DataFrame({"tta_condition": condition["tta_condition"],
                         "d_condition":  condition["d_condition"],
                         "t": model.t_domain(),
                         "rt_corr_distr": (sol.cdf_corr() if kind=="cdf" else sol.pdf_corr())/sol.prob_correct()})

def initialize_model(model_no, param_set):
    # all three models use Gaussian non-decision time
    overlay = models.OverlayNonDecisionGaussian(ndt_location=param_set.ndt_location, ndt_scale=param_set.ndt_scale)

    # model 1 uses collapsing bound, the other two use static bound
    if model_no == 1:
        bound = models.BoundCollapsingTta(b_0=param_set.b_0, k=param_set.k, tta_crit=param_set.tta_crit)
    else:
        bound = ddm.BoundConstant(B=param_set.b)

    # model 3 uses static drift, the other two use dynamic drift
    if model_no == 3:
        drift = models.DriftTtaDistanceStatic(alpha=param_set.alpha, beta=param_set.beta, theta=param_set.theta)
    else:
        drift = models.DriftTtaDistanceDynamic(alpha=param_set.alpha, beta=param_set.beta, theta=param_set.theta)

    model = ddm.Model(name="Model %i" % model_no, drift=drift, bound=bound, overlay=overlay,
                      noise=ddm.NoiseConstant(noise=1), T_dur=models.ModelStaticDriftFixedBounds.T_dur)

    return model

def simulate_model(model_no, param_set, conditions, ret="measures"):
    """
    Set ret to "measures" or "rt_cdf" or "rt_pdf" for saving p_turn and mean RT or RT CDF or RT PDF
    """
    model = initialize_model(model_no, param_set)

    if ret=="measures":
        sim_result = pd.DataFrame([get_model_measures(model, condition) for condition in conditions],
                                  columns=["tta_condition", "d_condition", "is_go_decision", "RT_go", "RT_stay"])
    else:
        sim_result = pd.concat([get_model_rt_distr(model, condition, kind=ret[-3:]) for condition in conditions])
    sim_result["subj_id"] = param_set.subj_id
    return sim_result

In [ ]:
def save_sim_results(model_no, file_name, conditions=None, cross_validation=False, ret="measures"):
    path = "../model_fit_results/model_%i/best_fit_parameters/" % model_no
    file_path = os.path.join(path, file_name)
    parameters = pd.read_csv(file_path).rename(columns={"tta": "tta_condition", "d": "d_condition"})

    # print(parameters)

    sim_results = [simulate_model(model_no, param_set,
                                  [param_set[["tta_condition", "d_condition"]].to_dict()] if cross_validation else conditions,
                                  ret=ret)
                   for idx, param_set in parameters.iterrows()]

    sim_results = pd.concat(sim_results)
    sim_results.to_csv(os.path.join("../model_fit_results/model_%i/simulation_results/" % model_no,
                                    file_name.replace("parameters_fitted", ret)),
                       index=False)

# Simulate model trained on full data

In [ ]:
conditions = [{"tta_condition": tta, "d_condition": d}
                       for tta in np.linspace(5,7,9)
                       for d in np.linspace(60,100,9)]

In [ ]:
save_sim_results(1, "subj_all.csv", conditions, cross_validation=False, ret="measures")

FileNotFoundError: [Errno 2] No such file or directory: '../model_fit_results/model_1/best_fit_parameters/subj_all.csv'